### Importing libraries

In [1]:
from ImgOps import *
import copy as cp
from sklearn.cluster import DBSCAN

### Imreading images

In [2]:
query_img = cv.imread('query_4.jpg')
query_img_gray = cv.cvtColor(query_img, cv.COLOR_BGR2GRAY)
# imshow(query_img)

train_img = cv.imread('train_4.jpg')
train_img_gray = cv.cvtColor(train_img, cv.COLOR_BGR2GRAY)
# imshow(train_img)

### Adding keypoints via SIFT

In [3]:
# Initiate SIFT detector
sift = cv.SIFT_create()

# find the keypoints and descriptors with SIFT
query_kps, query_des = sift.detectAndCompute(query_img_gray, None)
train_kps, train_des = sift.detectAndCompute(train_img_gray, None)

### Keypoints matching via FLANN

In [4]:
index_params = dict(algorithm=1, trees=5)
search_params = dict(checks=50)
flann = cv.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(query_des, train_des, k=2)
matchesMask = [[0, 0] for _ in range(len(matches))]

good_matches_query, good_matches_train = [], []
for i, (m, n) in enumerate(matches):
    if m.distance < .5 * n.distance:
        matchesMask[i] = [1, 0]
        good_matches_query.append(m)
#         good_matches_train.append(n)
good_matches_query = np.asarray(good_matches_query)
# good_matches_train = np.asarray(good_matches_train)

draw_params = dict(matchColor=(0, 255, 0),
                   matchesMask=matchesMask,
                   flags=cv.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
matched_img = cv.drawMatchesKnn(query_img, query_kps, train_img, train_kps, matches, None, **draw_params)

# imshow(matched_img)

### Clustering via DBSCAN

In [5]:
# Dictionary(ptQuery_ptTrain) = { (x_q, y_q): (x_t), (y_t) }
ptQuery_ptTrain = {}

for DMatch_q, DMatch_t in zip(good_matches_query, good_matches_query):
    pt_q = query_kps[DMatch_q.queryIdx].pt
    pt_t = train_kps[DMatch_q.trainIdx].pt
    ptQuery_ptTrain[pt_q] = pt_t


# Clustering
clusterized = DBSCAN(eps=50, min_samples=5).fit_predict(list(ptQuery_ptTrain.keys()))

# Dictionary(cluster_pts_q) = { cluster_name: [(x_q, y_q), ...] }
cluster_pts_q = {}
for gp, pt in zip(clusterized, list(ptQuery_ptTrain.keys())):
    if gp == -1: continue
    else:
        if gp not in cluster_pts_q:
            cluster_pts_q[gp] = [pt]
        else:
            cluster_pts_q[gp].append(pt)

cluster_pts_t = cp.deepcopy(cluster_pts_q)
for cluster in cluster_pts_t:
    for i, pt in enumerate(cluster_pts_t[cluster]):
        cluster_pts_t[cluster][i] = ptQuery_ptTrain[pt]

### Clusters' visualization

In [6]:
# for pts in cluster_pts_q.values():
#     for x, y in pts:
#         clust_img = cv.circle(query_img, (int(x), int(y)), radius=3, color=(255, 0, 255), thickness=-1)
#     imshow(clust_img)
#     clust_img = None

In [7]:
ptQuery_ptTrain

{(151.68614196777344, 270.49859619140625): (43.5200309753418,
  113.31671905517578),
 (152.91102600097656, 267.06561279296875): (49.264671325683594,
  95.2537841796875),
 (160.05548095703125, 270.32208251953125): (81.54954528808594,
  112.10921478271484),
 (286.5384826660156, 262.591796875): (21.16454315185547, 27.385150909423828),
 (288.956787109375, 307.5103759765625): (28.471757888793945,
  147.9222412109375),
 (289.62890625, 282.8891296386719): (30.79103660583496, 81.86742401123047),
 (292.1750793457031, 262.64990234375): (36.52845001220703, 27.28488540649414),
 (294.7551574707031, 294.5662536621094): (43.5200309753418,
  113.31671905517578),
 (296.6008605957031, 287.8304138183594): (49.264671325683594,
  95.2537841796875),
 (298.2112121582031, 269.9421081542969): (52.5528450012207, 46.70505905151367),
 (298.3581237792969, 262.30303955078125): (53.123756408691406,
  26.2194766998291),
 (299.8431701660156, 294.4839782714844): (58.1783332824707,
  113.04090118408203),
 (303.279693603

In [8]:
cluster_pts_q

{0: [(286.5384826660156, 262.591796875),
  (288.956787109375, 307.5103759765625),
  (289.62890625, 282.8891296386719),
  (292.1750793457031, 262.64990234375),
  (294.7551574707031, 294.5662536621094),
  (296.6008605957031, 287.8304138183594),
  (298.2112121582031, 269.9421081542969),
  (298.3581237792969, 262.30303955078125),
  (299.8431701660156, 294.4839782714844),
  (303.2796936035156, 266.1802978515625),
  (308.72174072265625, 294.2988586425781),
  (310.0554504394531, 266.3790283203125),
  (320.2637939453125, 275.2845153808594),
  (339.0787353515625, 264.3700256347656)],
 1: [(575.6551513671875, 140.9866943359375),
  (580.515625, 190.17245483398438),
  (586.98291015625, 172.07022094726562),
  (596.8635864257812, 163.81593322753906),
  (598.7796020507812, 134.25709533691406),
  (605.5413208007812, 103.8653335571289)],
 2: [(655.5125122070312, 64.98111724853516),
  (655.6220703125, 57.96885299682617),
  (661.877685546875, 81.86820220947266),
  (662.9232788085938, 53.468570709228516),

In [9]:
cluster_pts_t

{0: [(21.16454315185547, 27.385150909423828),
  (28.471757888793945, 147.9222412109375),
  (30.79103660583496, 81.86742401123047),
  (36.52845001220703, 27.28488540649414),
  (43.5200309753418, 113.31671905517578),
  (49.264671325683594, 95.2537841796875),
  (52.5528450012207, 46.70505905151367),
  (53.123756408691406, 26.2194766998291),
  (58.1783332824707, 113.04090118408203),
  (66.34867095947266, 36.381103515625),
  (81.54954528808594, 112.10921478271484),
  (84.62295532226562, 36.58852005004883),
  (112.34420013427734, 59.89463424682617),
  (162.92996215820312, 29.234397888183594)],
 1: [(19.76636505126953, 45.790496826171875),
  (25.23940658569336, 140.3516845703125),
  (21.62360954284668, 138.45994567871094),
  (49.264671325683594, 95.2537841796875),
  (52.78783416748047, 36.486366271972656),
  (52.54483413696289, 41.493587493896484)],
 2: [(9.529679298400879, 27.013498306274414),
  (9.646004676818848, 19.929960250854492),
  (15.882925987243652, 43.789405822753906),
  (17.116510

In [10]:
# for cluster in cluster_pts_q:
#     print(f'Cluster #{cluster}:\nSize of query cluster = {len(cluster_pts_q[cluster])}\nSize of train cluster = {len(cluster_pts_t[cluster])}', end='\n\n\n')

### Feature Homography

In [11]:
# src_pts = np.float32(cluster_pts_t[2]).reshape(-1, 1, 2)
# dst_pts = np.float32(cluster_pts_q[2]).reshape(-1, 1, 2)

# M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.)
# matchesMask = mask.ravel().tolist()

# h, w, d = train_img.shape
# pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
# dst = cv.perspectiveTransform(pts, M)

# res_img = cv.polylines(query_img, [np.int32(dst)], True, 255, 3, cv.LINE_AA)

In [12]:
# imshow(res_img)

In [13]:
for cluster in cluster_pts_q:
    print(f'NEW CLUSTER! {cluster}')
    
    src = np.float32(cluster_pts_t[cluster]).reshape(-1, 1, 2)
    dst = np.float32(cluster_pts_q[cluster]).reshape(-1, 1, 2)
    print(f'SRC_PTS: {src}\n -----\n DST_PTS: {dst}\n\n')
    
    M, _ = cv.findHomography(src, dst, cv.RANSAC, 5.)
    print(f'M = {M}, type-M = {type(M)}\n\n')
    
    h, w = train_img_gray.shape
    print(f'h = {h}, w = {w}\n\n')
    
    pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
    print(f'PTS = {pts}, type-PTS = {type(pts)}\n\n\n\n\n')
    
    dst = cv.perspectiveTransform(pts, M)
    
    print(f'DST FOR TRANSFORM{dst}')
    
    res_img = cv.polylines(query_img, [np.int32(dst)], True, 255, 3, cv.LINE_AA)

NEW CLUSTER! 0
SRC_PTS: [[[ 21.164543  27.38515 ]]

 [[ 28.471758 147.92224 ]]

 [[ 30.791037  81.867424]]

 [[ 36.52845   27.284885]]

 [[ 43.52003  113.31672 ]]

 [[ 49.26467   95.253784]]

 [[ 52.552845  46.70506 ]]

 [[ 53.123756  26.219477]]

 [[ 58.178333 113.0409  ]]

 [[ 66.34867   36.381104]]

 [[ 81.549545 112.109215]]

 [[ 84.622955  36.58852 ]]

 [[112.3442    59.894634]]

 [[162.92996   29.234398]]]
 -----
 DST_PTS: [[[286.53848 262.5918 ]]

 [[288.9568  307.51038]]

 [[289.6289  282.88913]]

 [[292.17508 262.6499 ]]

 [[294.75516 294.56625]]

 [[296.60086 287.8304 ]]

 [[298.2112  269.9421 ]]

 [[298.35812 262.30304]]

 [[299.84317 294.48398]]

 [[303.2797  266.1803 ]]

 [[308.72174 294.29886]]

 [[310.05545 266.37903]]

 [[320.2638  275.28452]]

 [[339.07874 264.37003]]]


M = [[ 3.87612914e-01 -1.23916839e-02  2.78632394e+02]
 [ 2.05053731e-02  3.61853927e-01  2.52244918e+02]
 [ 4.90263309e-05 -3.21145161e-05  1.00000000e+00]], type-M = <class 'numpy.ndarray'>


h = 180

In [15]:
imshow(res_img)